In [1]:
from aips import get_engine, set_engine
from aips.spark.dataframe import from_sql
from aips.spark import create_view_from_collection
import tqdm

set_engine("opensearch")
engine = get_engine()

In [2]:
#%run chapters/ch04/1.setting-up-the-retrotech-dataset.ipynb

In [3]:
def transform_event(signal):
    return {"application": "aips_search",
            "action_name": signal["type"],
            "query_id": signal["query_id"], 
            "client_id": signal["user"],
            "timestamp": datetime.now(),
            "message_type": signal["type"],
            "message": signal["target"],
            "target": signal["target"]}   

In [4]:
def get_events_dataframe():
    signals_collection = engine.get_collection("signals")
    create_view_from_collection(signals_collection, "signals")
    query = """SELECT type AS action_name, query_id, user AS client_id
                      signal_time AS timestamp, type AS message_type,
                      target AS message, target AS target
               FROM signals WHERE type != 'query'"""
    events = from_sql(query)
    return events

In [5]:
#events_collection = engine.create_collection("ubi_events")
#ubi_events_dataframe = get_events_dataframe()
#events_collection.write(ubi_events_dataframe)

In [ ]:
def get_queries_dataframe():
    signals_collection = engine.get_collection("signals")
    create_view_from_collection(signals_collection, "signals")
    queries = from_sql("SELECT * FROM signals WHERE type = 'query'")
    queries_transformed = queries.rdd.map(lambda r: 
        (r["signal_time"], r["query_id"], r["user"], r["target"]))
    ubi_queries_dataframe = queries_transformed.toDF(
        ["timestamp", "query_id", "client_id", "user_query"])
    return ubi_queries_dataframe

In [6]:
def execute_search(collection, signal, log=False):
    request = {"query": signal["user_query"],
               "query_fields": ["name", "manufacturer",
                                "long_description", "short_description"],
               "return_fields": ["*"],
               "limit": 10,
               "ext": {"ubi": signal}}
    results = collection.search(**request, log=log)
    return results

In [8]:
queries_collection = engine.create_collection("ubi_queries")
ubi_queries_dataframe = get_queries_dataframe()
queries_collection.write(ubi_queries_dataframe)

Wiping "ubi_queries" collection
Creating "ubi_queries" collection
Successfully written 725459 documents


In [7]:

products_collection = engine.get_collection("products")
ubi_queries_dataframe = get_queries_dataframe()
for q in tqdm.tqdm(ubi_queries_dataframe.collect(), total=ubi_queries_dataframe.count()):
    execute_search(products_collection, q.asDict(), log=True)

  0%|          | 0/725459 [00:00<?, ?it/s]

Search Request:
{
  "query": {
    "query_string": {
      "query": "Iphone headphones",
      "boost": 0.454545454,
      "fields": [
        "name",
        "manufacturer",
        "long_description",
        "short_description"
      ]
    }
  },
  "size": 10,
  "fields": [
    "*"
  ],
  "ext": {
    "ubi": {
      "timestamp": 1572238882476,
      "query_id": "u420243_0_1",
      "client_id": "u420243",
      "user_query": "Iphone headphones"
    }
  }
}


ValueError: {'error': {'root_cause': [{'type': 'named_object_not_found_exception', 'reason': '[1:204] unknown field [ubi]'}], 'type': 'named_object_not_found_exception', 'reason': '[1:204] unknown field [ubi]'}, 'status': 400}

In [12]:
def write_events_to_collection(collection):
    ubi_events_collection = engine.get_collection("ubi_events")
    create_view_from_collection(ubi_events_collection, "ubi_events")
    events = from_sql("SELECT * FROM ubi_events")
    events_transformed = events.rdd.map(lambda r: 
        (r["timestamp"], r["query_id"], r["client_id"],
         r["message"], r["message_type"]))
    events_dataframe = events_transformed.toDF(
        ["signal_time", "query_id", "user", "target", "type"])
    collection.write(events_dataframe) 

def write_queries_to_collection(collection):
    ubi_queries_collection = engine.get_collection("ubi_queries")
    create_view_from_collection(ubi_queries_collection, "ubi_queries")
    queries = from_sql("SELECT * FROM ubi_queries")
    queries_transformed = queries.rdd.map(lambda r: 
        (r["timestamp"], r["query_id"], r["client_id"],
         r["user_query"], "query"))
    queries_dataframe = queries_transformed.toDF(
        ["signal_time", "query_id", "user", "target", "type"])
    collection.write(queries_dataframe)

signals_collection = engine.create_collection("signals")
write_queries_to_collection(signals_collection)
write_events_to_collection(signals_collection)

Wiping "signals" collection
Creating "signals" collection
Successfully written 725459 documents


ValueError: Some of types cannot be determined by the first 100 rows, please try again with sampling